## Elliptic Curve Example 

There are many great introductions to elliptic curves that cover the intuition and abstract ideas in great detail. Some links of these articles are noted at the bottom. 

In this note, we will focus on some abstract nonsense about elliptic curves then give an example of the group operation for a small elliptic curve. The computation of these group operation will be done with an naive direct approach since the group is small. 

- Millers Algorithm
- BLS12-381 Curve
- Pairing


##  General Abstract Nonsense

An elliptic curve over a finite field $K$ is the set of point in $K^2$ which solve the equation below: 

$$ y^2 = x^3 +ax +b $$

Suprisingly (given some more conditions), this set of points form a group when the point $O$, being the unique point at infinity, is added to the set. For finite fields of a prime order, the elliptic curve is itself a cyclic group of is isomorphic to the direct product of cyclic groups. There are stronger results on the classification of an Elliptic curves up to isomorphism but for an overview this gives a pretty nice guarantee. 


## Example Computation 


### 1. Checking whether points lay on E
We will now look at the elliptic curve over the field of order 101 defined by the coefficents: 

In [54]:
# Coefficents
a = 0
b = 3
# Order of Field
p = 101



Lets now look at the set of points in $K^2$ and try to detemine for a few examples which lay on our curve $E$. This could be done as our set is of order 101^2. However, it may not be to enlightening to look at. 

In [55]:
print('Is the point (0,1) on curve $E$: ',valid(Point(0,1)))
print('Is the point (1,1) on curve $E$: ',valid(Point(1,1)))
print('Is the point (1,2) on curve $E$: ',valid(Point(1,2)))
print('Is the point (2,2) on curve $E$: ',valid(Point(2,2)))
print('Is the point (2,1) on curve $E$: ',valid(Point(2,1)))
print('Is the point (3,7) on curve $E$: ',valid(Point(3,7)))

Is the point (0,1) on curve $E$:  False
Is the point (1,1) on curve $E$:  False
Is the point (1,2) on curve $E$:  True
Is the point (2,2) on curve $E$:  False
Is the point (2,1) on curve $E$:  False
Is the point (3,7) on curve $E$:  False


We can even pick a point at random and see if it is on our curve. We
could also look at the probability of how often the a random point lies on the curve but will leave it to the reader. 

In [56]:
from random import randint
x = randint(0,101)
y = randint(0,101)

print('Is the point ({},{}) on curve $E$: '.format(x,y),valid(Point(x,y)))

Is the point (15,42) on curve $E$:  False


### 2. Generating the Elliptic Curve Group


From abstract nonsense, we know that the elliptic curve will either be cylic or the the direct product of cyclic groups. As such either our curve will have 1 or 2 generators.

In [57]:
P = Point(1, 2)
#
#


def generate_cyclic_subgroup(g):
    """
    Given a element g generates the cyclic subgroup using bruteforce. 
    """
    hold = [g]
    Gen = g
    for i in range(1,p+1):
        g = ec_add(Gen,g)
        hold.append(g)
        if g == O:
            break
    return hold

        


ok = generate_cyclic_subgroup(P)
Table().with_column('Points',ok).show()

Points
"Point(x=1, y=2)"
"Point(x=68, y=74)"
"Point(x=26, y=45)"
"Point(x=65, y=98)"
"Point(x=12, y=32)"
"Point(x=32, y=42)"
"Point(x=91, y=35)"
"Point(x=18, y=49)"
"Point(x=18, y=52)"
"Point(x=91, y=66)"



Above is the elements in $E$ that are generated by the point(1,2). However question remains: is this all the elements within our set? 

Lets check by brute force.

In [58]:
tablecol = []
count =0
for i in range(0,101):
    for j in range(0,101):
        if valid(Point(i,j)) and Point(i,j) not in ok:
            tablecol.append(Point(i,j))
            count= count +1
print('Number Of Remaining Elements:',count)
Table().with_column("Remaining elements",tablecol)

Number Of Remaining Elements: 85


Remaining elements
[ 3 38]
[ 3 63]
[ 6 44]
[ 6 57]
[ 7 12]
[ 7 89]
[9 5]
[ 9 96]
[11 18]
[11 83]


This more or less tells us that our group is not generated by the point (1,2) and contains 102 elements. Lets see if we can find a generator

In [59]:
ok = generate_cyclic_subgroup(Point(6,44))
len(ok)

102

# Conclusions 


1. https://medium.com/@VitalikButerin/exploring-elliptic-curve-pairings-c73c1864e627
2. https://blog.statebox.org/elliptic-curve-pairings-213131769fac#:~:text=An%20elliptic%20curve%20pairing%20is,and%20integers%20a%20and%20b%20.
3. https://alinush.github.io/2022/12/31/pairings-or-bilinear-maps.html

In [8]:
#*Please note this code has been pulled from stackoverflow however the link has been lost. If this code looks recoginizable please let me know.*#


######################################################################
"""Here we are creating a point class to represent the affine points in  
   our space K^2. """
######################################################################
from collections import namedtuple
Point = namedtuple("Point", "x y")



######################################################################
"""Lets The point at infinity (origin for the group law)."""
######################################################################
O = 'Origin'



######################################################################
"""Determine whether we have a valid representation of a point
   on our curve.  We assume that the x and y coordinates
   are always reduced modulo p, so that we can compare
   two points for equality with a simple ==."""
######################################################################

def valid(P):
    if P == O:
        return True
    else:
        return ((P.y**2 - (P.x**3 + a*P.x + b)) % p == 0 and 0 <= P.x < p and 0 <= P.y < p)

    
    
######################################################################
    """
    Compute an inverse for x modulo p, assuming that x
    is not divisible by p.
    """
######################################################################
def inv_mod_p(x):
    if x % p == 0:
        raise ZeroDivisionError("Impossible inverse")
    return pow(x, p-2, p)

######################################################################
    """
    Inverse of the point P on the elliptic curve y^2 = x^3 + ax + b.
    """
######################################################################


def ec_inv(P):
    if P == O:
        return P
    return Point(P.x, (-P.y)%p)

######################################################################
    """
    Sum of the points P and Q on the elliptic curve y^2 = x^3 + ax + b.
    """
######################################################################

def ec_add(P, Q):
    if not (valid(P) and valid(Q)):
        raise ValueError("Invalid inputs")

    # Deal with the special cases where either P, Q, or P + Q is
    # the origin.
    if P == O:
        result = Q
    elif Q == O:
        result = P
    elif Q == ec_inv(P):
        result = O
    else:
        # Cases not involving the origin.
        if P == Q:
            dydx = (3 * P.x**2 + a) * inv_mod_p(2 * P.y)
        else:
            dydx = (Q.y - P.y) * inv_mod_p(Q.x - P.x)
        x = (dydx**2 - P.x - Q.x) % p
        y = (dydx * (P.x - x) - P.y) % p
        result = Point(x, y)

    # The above computations *should* have given us another point
    # on the curve.
    assert valid(result)
    return result



In [ ]:
# Defining our Elliptic Curve over the field of order 101
a = 0
b = 3
p = 101
#
